In [ ]:

EPOCHS = 10
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f"runs/piece_to_move_{timestamp}")

train_data, validation_data = torch.utils.data.random_split(train_data, [1-TEST_PERCENT, TEST_PERCENT])

training_loader = torch.utils.data.dataLoader(train_data, BATCH_SIZE, shuffle=True, pin_memory=True)
validation_loader = torch.utils.data.dataLoader(validation_data, BATCH_SIZE, shuffle=True, pin_memory=True)

def train_one_epoch(epoch_index: int, tb_writer, optimizer, training_loader, loss_fn): 
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        inputs = data[0]
        labels = data[1]

        optimizer.zero_grad()
        outputs = piece_to_move_net(inputs)

               
        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999: 
            last_loss = running_loss / 1000
            print(f" batch {i + 1}, loss: {last_loss}")
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar("Loss/train", last_loss, tb_x)
            running_loss = 0.

    return last_loss


def train_multiple_epochs(n_epochs, model, writer, validation_loader, loss_fn):
    for epoch in range(EPOCHS): 
        print(f"EPOCH {epoch + 1}: ")

        model.train(True)
        avg_loss = train_one_epoch(epoch, writer)

        model.train(False)
        running_vloss = 0.0
        for i, v_data in enumerate(validation_loader):
            vinputs = v_data[0]
            vlabels = v_data[1]

            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print(f"LOSS train {avg_loss}, valid {avg_vloss}")

        writer.add_scalars("Training vs Validation Loss", {
            "Training": avg_loss, "Validation": avg_vloss}, 
            epoch + 1)
        writer.flush()
        
        if avg_vloss < best_vloss:
            best_vloss = avg_loss
            model_path = f"model_{timestamp}_{epoch}"
            torch.save(model.state_dict(), model_path)

        epoch += 1
